# 第5回課題：tf-idfを使って「銀河鉄道の夜」の重要語を抽出しよう

<img src="2019-07-15 12.50.02.png"><br/>
<img src="2019-07-15 12.49.37.png"><br/>
<img src="2019-07-15 12.49.11.png">



## 入力データの説明

`texts/novels_miyazawa_wakati.json`には、宮沢賢治の全小説の分かち書き文がjson形式で記録されています。   
記録形式は以下の通りです。

```
[
    {
        "author": "宮沢賢治",
        "title": "『春と修羅』",
        "text": "目次 \n『 春 と 修羅 』 \n春 と...(本文続く）
    },
    ...
]
```
`title`が`銀河鉄道の夜`であるような要素の`text`の値には「銀河鉄道の夜」の本文の分かち書き文が納められています。

## 課題１：tf-idfモデルの学習

宮沢賢治の全作品を使って、各作品を1文書としたときのtf-idfを学習してください。   
tf-idfの学習には、`sklearn.feature_extraction.text`の`TfidfVectorizer`を使ってください。   
ここで、TfidfVectorizerのパラメータは以下のようにしてください。
-    max_features=1000 (語彙サイズは最大1000個)
-    max_df=5　（5つ以下の小説までに現れる語彙）
-    min_df=3　（3つ以上の小説に現れる語彙）

## 課題２：「銀河鉄道の夜」の重要語の抽出

小説「銀河鉄道の夜」のtf-idf値が大きい単語上位10単語とそのtf-idf値を、   
各行にカンマ区切りで単語とtf-idf値が並んだ形式で、ex5-tfidf.txtという名前のファイルに出力してください。

## ヒント

`TfidfVectorizer`には、各要素が1つの小説の本文全体（単語が半角空文字で区切られた分かち書き文）であるようなリストを渡します。   
今回入力とするjsonファイルには249作品が納められているので、`TfidfVectorizer`に渡す行列は249個の要素を持つのベクトルということになります。   
このようなリストは以下のプログラムで作成することができます。

以下のプログラムでは、

1. `texts/novels_miyazawa_wakati.json`の本文（jsonファイルを辞書として読み込んだ時の各要素のキーが`text`）を読み込み
'\n'を空白に置換（replace('\n', ' '))した後で、`wakati`という名前のリストに追加(append)しています。   
つまり、`wakati`の$i$番目の要素は、$i$番目に登録されている小説の本文（全文の分かち書き文）というわけです。   
`novels_miyazawa_wakati.json`には249作品が納められているので、このリストは249次元となります。   

2. また同時に、`novels_miyazawa_wakati.json`において、ある作品名の小説が何番目に登録されているかを返す辞書`title2ID`を生成しています。   
つまり、`title2ID['作品名'] = [登場順]`です。   
たとえば「銀河鉄道の夜」は0番目から数えて225番目に現れるので、  
    `title2ID['銀河鉄道の夜'] = 225`  
となります。


In [1]:
############################################
### このセルは変更しないでください！！！ ###
############################################

import json
import numpy as np

file = 'texts/novels_miyazawa_wakati.json'

with open(file, 'r', encoding='utf-8') as fi:
    novels = json.load(fi)

print('小説の数:', len(novels))#249

wakati = [] #novels_miyazawa_wakati.jsonには249作品が納められているので、このリストは249次元となります。
num = 0
title2ID = {}
#text = jsonファイルを辞書として読み込んだ時の各要素のキー
for novel in novels:
    wakati.append(novel['text'].replace('\n',' ')) #i番目に登録されている小説の本文（全文の分かち書き文）
    title2ID[novel['title']] = num#ある作品名の小説が何番目に登録されているかを返す辞書title2IDを生成
    num += 1
    
print(title2ID['銀河鉄道の夜']) #225


小説の数: 249
225


## 課題１：tf-idfモデルの学習

以下にtf-idfを学習するプログラムを書いてください。   
上のヒントを使ってもいいし使わなくてもいいです。

`max_features=1000 (語彙サイズは最大1000個)`
という条件が指定されていますから、
`vectorizer = TfidfVectorizer(max_features=1000, max_df=5, min_df=3)`
となります。
また、学習をする
`X = vectorizer.fit_transform(wakati)`
の行が必要です。

後半は以下のようにすればOKです。
`
    feature_names = vectorizer.get_feature_names() `
1000種類の単語のリストを獲得します
単語とそのtf-idf値の対を辞書として登録
`pair = dict(zip(feature_names, X[title2ID['銀河鉄道の夜'],:].toarray()[0]))`
tf-idfの高い順にソートして、単語とtf-idfの対をタプルとしてリスト化し、辞書に代入する
`dic = [(x, pair[x]) for x in sorted(pair, key=lambda x:-pair[x])]`

結果の出力には以下を使ってください。
`with open('ex5-tfidf.txt', 'w', encoding='utf-8') as f:`
`for d in dic[:10]:`
`f.write(d[0]+','+str(round(d[1],3))+'\n')`

In [2]:
#TfidfVectorizerには、各要素が1つの小説の本文全体（単語が半角空文字で区切られた分かち書き文）であるようなリストを渡します。
#今回入力とするjsonファイルには249作品が納められているので、TfidfVectorizerに渡す行列は249個の要素を持つのベクトルということになります。

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, max_df=5, min_df=3)
X = vectorizer.fit_transform(wakati)

feature_names = vectorizer.get_feature_names()#1000種類の単語のリストを獲得
pair = dict(zip(feature_names, X[title2ID['銀河鉄道の夜'],:].toarray()[0]))
dic = [(x, pair[x]) for x in sorted(pair, key=lambda x:-pair[x])]

with open('ex5-tfidf.txt', 'w', encoding='utf-8') as f:
    for d in dic[:10]:
        f.write(d[0]+','+str(round(d[1],3))+'\n')  

## 課題２：「銀河鉄道の夜」の重要語の抽出

小説「銀河鉄道の夜」のtf-idf値が大きい単語上位10単語とそのtf-idf値を、   
**各行にカンマ区切りで単語とtf-idf値が並んだ形式で、`ex5-tfidf.txt`という名前のファイルに出力**してください。

ex5-tfidf.txt`は以下のようにります。
```
神さま,0.447
切符,0.289
牛乳,0.263
ぱいに,0.237
十字架,0.22
向う岸,0.22
白鳥,0.22
さそり,0.202
烏瓜,0.192
もろこし,0.165
```

	
`TopicAnalysis2.ipynb`の内容を使えばできるはずです。
ただし、`TopicAnalysis2.ipynb`では文書数(Categories)は6個でしたが、課題では文書数len(novels)は249（0～248)です。
Xには249個の小説それぞれの語彙1000単語のtf-idf値が並んでいるので、「銀河鉄道の夜」（225番目）のtf-idf値は
     
`X[225,:].toarray()[0]`

  でとってくることができます。<br/>
`pair = dict(zip(feature_names, X[225,:].toarray()[0]))`

   とすることで、「銀河鉄道の夜」における、各語彙とそのtf-idf値のペアを生成することができます。
あとは、pairのうちtf-idfが高いものから順に10個をとってくればOKです。

In [36]:
dic = {}
#単語とそのtf-idf値の対を辞書として登録
#語彙1000単語のtf-idf値が並んでいる
pair = dict(zip(feature_names, X[225,:].toarray()[0]))#各語彙とそのtf-idf値のペアを生成
# tf-idfの高い順にソートして、単語とtf-idfの対をタプルとしてリスト化し、辞書に代入する
dic.update([(x, pair[x]) for x in sorted(pair, key=lambda x:-pair[x])])
result = {k:d[k] for k in list(d)[:3]}
#result

In [35]:
result_dic = {k:dic[k] for k in list(dic)[:10]}
#esult_dic

In [38]:
for k in list(dic)[:10]:
    print(k+',''{:.4g}'.format(dic[k]))

神さま,0.4102
切符,0.2654
牛乳,0.2413
ぱいに,0.2172
十字架,0.2018
向う岸,0.2018
白鳥,0.2018
さそり,0.1859
烏瓜,0.1766
もろこし,0.1514
